In [4]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_squared_error


/Users/nickr/opt/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [5]:
!pip install nba_api

from nba_api.stats.static import teams
from nba_api.stats.endpoints import teamgamelog, playergamelog,leaguegamefinder
from nba_api.live.nba.endpoints import boxscore
from nba_api.stats.endpoints import boxscoretraditionalv2
from nba_api.stats.static import players
from nba_api.stats.endpoints import commonplayerinfo
from nba_api.stats.endpoints import playerfantasyprofile
from nba_api.stats.endpoints import commonteamroster
from nba_api.stats.endpoints import TeamYearByYearStats

In [106]:
# Set the timeout value (in seconds)
timeout_value = 60  # Adjust this value as needed


In [153]:
# # Get LeBron's game log data
# Lebron_id = 2544
# gamelog_lebron = playergamelog.PlayerGameLog(player_id=Lebron_id, season=2023)
# gamelog_data_lebron = gamelog_lebron.get_data_frames()[0]

# # Get Giannis's game log data
# giannis_id = 203507
# gamelog_giannis = playergamelog.PlayerGameLog(player_id=giannis_id, season=2023)
# gamelog_data_giannis = gamelog_giannis.get_data_frames()[0]

# MPJ_ID = 1629008
# gamelog_MPJ = playergamelog.PlayerGameLog(player_id=MPJ_ID, season=2023)
# gamelog_data_MPJ = gamelog_MPJ.get_data_frames()[0]

# Josh_hart_id = 1628404
# gamelog_hart = playergamelog.PlayerGameLog(player_id=Josh_hart_id, season=2023)
# gamelog_data_hart = gamelog_hart.get_data_frames()[0]

# Brunson_id = 1628973
# gamelog_brunson = playergamelog.PlayerGameLog(player_id=Brunson_id, season=2023)
# gamelog_data_brunson = gamelog_brunson.get_data_frames()[0]


# Jokic_id = 203999
# gamelog_Jokic = playergamelog.PlayerGameLog(player_id=Jokic_id, season=2023)
# gamelog_data_Jokic = gamelog_Jokic.get_data_frames()[0]

# Ayton_id = 1629028
# gamelog_ayton = playergamelog.PlayerGameLog(player_id=Ayton_id, season=2023)
# gamelog_data_ayton = gamelog_ayton.get_data_frames()[0]

import requests



player_ids = [
    2544,   # LeBron James
    201939, # Stephen Curry
    203076, # Anthony Davis
    2546,   # Russell Westbrook
    203507, # Giannis Antetokounmpo
    1629029,# Luka Dončić
    201566, # James Harden
    203954, # Jayson Tatum
    1629027,# Trae Young
    203954, # Damian Lillard
    203999, # Zion Williamson
    1629029,# LaMelo Ball
    202331, # Chris Paul
    1629027,# Donovan Mitchell
    1628369,# Bam Adebayo
    203076, # Nikola Jokić
    1629029,# Zion Williamson
    203506, # Ben Simmons
    1629027,# De'Aaron Fox
    201588, # DeMar DeRozan
    1629029,# Ja Morant
    203507, # Pascal Siakam
    202691, # Kyrie Irving
    1629027,# Deandre Ayton
    1629029, # Karl-Anthony Towns
    1626220, #royce O'neal
    1628464, #daniel theis
    1628401, #derrick white
    1627827, #dorian finney smith
    1629023, #pj washington
    1628969, #mikal bridges
    1629001, #deanthony melton
    1626196, #josh richardson
    203469, #cody zeller
    1627826, #ivica zubac
]



game_log_combined = []
for player_id in player_ids:
    gamelog_player = playergamelog.PlayerGameLog(player_id=player_id, season=2023,timeout=timeout_value)
    gamelog_data_player = gamelog_player.get_data_frames()[0]
    game_log_combined.append(gamelog_data_player)

game_log_combined = pd.concat(game_log_combined, ignore_index=True)



In [154]:
# combined_data = pd.concat([gamelog_data_lebron, gamelog_data_giannis, gamelog_data_MPJ, gamelog_data_hart, gamelog_data_brunson,gamelog_data_ayton,gamelog_data_Jokic], ignore_index=True)
print(game_log_combined.keys())

Index(['SEASON_ID', 'Player_ID', 'Game_ID', 'GAME_DATE', 'MATCHUP', 'WL',
       'MIN', 'FGM', 'FGA', 'FG_PCT', 'FG3M', 'FG3A', 'FG3_PCT', 'FTM', 'FTA',
       'FT_PCT', 'OREB', 'DREB', 'REB', 'AST', 'STL', 'BLK', 'TOV', 'PF',
       'PTS', 'PLUS_MINUS', 'VIDEO_AVAILABLE'],
      dtype='object')


In [155]:
# Define features and target
# features = ['PTS', 'AST', 'REB','PLUS_MINUS','MIN','TOV','PF','FG_PCT']  # Example features
# target = ['PTS','AST','TOV']  # Predicting points scored
features = ['AST','REB','PLUS_MINUS','MIN','TOV','PF','FG_PCT','FTA','FG3_PCT']  # Example features
target = ['PTS']  # Predicting points scored

X = game_log_combined[features]
y = game_log_combined[target]

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Choose a model and train it
rf_model = RandomForestRegressor()
rf_model.fit(X_train, y_train)

/var/folders/pq/8mm0wb857m5cqlp3f71p96q40000gn/T/ipykernel_8510/212494968.py:15: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rf_model.fit(X_train, y_train)


RandomForestRegressor()

In [156]:
# Make predictions
y_pred = rf_model.predict(X_test)

# Evaluate the model
r2 = r2_score(y_test, y_pred)
print("R^2 Score:", r2)

R^2 Score: 0.8839247751214221


In [161]:
# Assume you have the feature names used during training stored in a list called 'features'

Dame_id=1630162
gamelog_dame = playergamelog.PlayerGameLog(player_id=Dame_id, season=2023)
gamelog_data_dame = gamelog_dame.get_data_frames()[0]
latest_dame_stats = gamelog_data_dame.iloc[-10][features].values
latest_dame_points = gamelog_data_dame.iloc[-10][target].values


# print(">> TESTING X-VALUES")
# print(X_test)
# print(">> TESTING y-VALUES")
# print(y_test)

print(">> LAST GAME STATS")
print(latest_dame_stats)
X_pred = pd.DataFrame([latest_dame_stats], columns=features) #formatting
print(">> PREDICTED X-VALUES")
print(X_pred)
print(">> EXPECTED Y-VALUE")
print(latest_dame_points)
# Make predictions using the trained model
predicted_points = rf_model.predict(X_pred)
print(">> PREDICTED Y-VALUES")
print(predicted_points)
# Print the predicted points for LeBron's next game
print("Predicted stats for Dames next game... PTS:", predicted_points[0])
# not predicting next game/ always predicts same as input

>> LAST GAME STATS
[4 4 -9 36 5 1 0.438 5 0.5]
>> PREDICTED X-VALUES
   AST  REB  PLUS_MINUS  MIN  TOV  PF  FG_PCT  FTA  FG3_PCT
0    4    4          -9   36    5   1   0.438    5      0.5
>> EXPECTED Y-VALUE
[20]
>> PREDICTED Y-VALUES
[25.44]
Predicted stats for Dames next game... PTS: 25.44


In [149]:
X_train

,AST,REB,PLUS_MINUS,MIN,TOV,PF,FG_PCT,FTA,FG3_PCT
196,2,14,-6,35,2,2,0.636,3,1.000
1197,12,14,-4,39,4,2,0.609,7,0.333
1292,0,2,5,27,0,0,0.400,0,0.222
81,3,4,6,30,2,1,0.429,8,0.455
1439,10,6,13,41,5,1,0.393,8,0.214
...,...,...,...,...,...,...,...,...,...
1130,11,11,11,40,4,2,0.500,4,0.538
1294,1,1,-1,31,1,5,0.357,4,0.286
860,8,11,27,35,1,1,0.450,11,0.143
1459,4,6,-21,35,5,3,0.429,8,0.000


In [150]:
y_train

,PTS
196,32
1197,34
1292,14
81,31
1439,32
...,...
1130,35
1294,16
860,29
1459,13


In [94]:
#new model
import dill as pickle
import matplotlib.pyplot as plt


In [152]:
pickle.dump(rf_model, open('./saved_models/full_pipeline.pkl', 'wb'))
